## the cube
this will make it easy to "roll" the cube around

In [124]:
class Cube():
    def __init__(self) -> None:
        self.top = None
        self.bottom = None
        self.left = None
        self.right = None
        self.front = None
        self.back = None

    def roll(self, direction):

        #roll forward
        if direction == 0:
            temp = self.bottom
            self.bottom = self.front
            self.front = self.top
            self.top = self.back
            self.back = temp

        #roll right
        if direction == 1:
            temp = self.bottom
            self.bottom = self.right
            self.right = self.top
            self.top = self.left
            self.left = temp

        #roll back
        if direction == 2:
            temp = self.bottom
            self.bottom = self.back
            self.back = self.top
            self.top = self.front
            self.front = temp
        
        #roll left
        if direction == 3:
            temp = self.bottom
            self.bottom = self.left
            self.left = self.top
            self.top = self.right
            self.right = temp

    def __str__(self):
        return f"""
        \t{self.front}
        {self.left}\t{self.top}\t{self.right}
        \t{self.back}
        \t{self.bottom}
        """

## the state
this will keep track of the cube, score, and N as the cube moves around

In [125]:
class PlayerState():
    def __init__(self) -> None:
        self.cube = Cube()

        self.N = 0
        self.score = 0
        self.x = 0
        self.y = 0

        self.history = [(0,0)]

    def move(self, d, dir):
        self.N += 1
        self.cube.roll(d)
        self.x += dir[0]
        self.y += dir[1]

        self.history.append((self.x, self.y))
        

## the grid

this just makes it easier to have the bottom left as 0,0

In [126]:
class Grid():
    def __init__(self, grid) -> None:
        self.grid = grid
        self.rows = len(self.grid)
        self.cols = len(self.grid[0])

    def get(self, x, y):
        return self.grid[self.rows - 1 - y][x]

    def set(self, x, y, v):
        self.grid[self.rows - 1 - y][x] = v

    def valid(self, x, y):
        if x<0 or x>=self.cols:
            return False

        if y<0 or y>=self.rows:
            return False
        
        return True

    def __str__(self):
        os = ""
        for row in self.grid:
            r = ""
            for item in row:
                r += "\t" + str(item)
            os += r + '\n'
        return os

In [127]:
grid = Grid(
    [
    [  57,  33, 132, 268, 492, 732],
    [  81, 123, 240, 443, 353, 508],
    [ 186,  42, 195, 704, 452, 228],
    [  -7,   2, 357, 452, 317, 395],
    [   5,  23,  -4, 592, 445, 620],
    [   0,  77,  32, 403, 337, 452]
    ]
)

## DFS

Roll our cube around checking every path. If the score of the move matches the tile, keep going.
Our cube is blank to start so if the top face is blank when we reach a tile, just set the face
equal to whatever it needs to be for the roll to be valid.

In [128]:
dirs = ((0,1), (1,0), (0,-1), (-1,0))
import copy


def rec(state):
    #did we win?
    if state.x == grid.cols-1 and state.y == grid.rows-1:
        return state
    
    #dfs
    for d, dir in enumerate(dirs):
        dx, dy = dir
        nx, ny = state.x + dx, state.y + dy
        
        if not grid.valid(nx, ny):
            continue

        new_state = copy.deepcopy(state)
        new_state.move(d, dir)

        #in order to move to this tile, what would the top face have to be?
        tile_val = grid.get(nx, ny)
        diff = tile_val - new_state.score
        pot_face = diff / new_state.N

        # if its not a whole number, skip
        if not pot_face.is_integer():
            continue

        # if the top face isnt the right number, skip
        if new_state.cube.top is not None and new_state.cube.top != pot_face:
            continue

        # top face is either none or the right number
        # if none, set it to the right number
        new_state.score = tile_val
        if new_state.cube.top == None:
            new_state.cube.top = pot_face

        # sanity check
        assert pot_face == new_state.cube.top

        ret = rec(new_state)
        if ret is not None:
            return ret

ans = rec(PlayerState())

# the answer

In [133]:
import numpy as np

grid2 = copy.deepcopy(grid)
for x,y in ans.history:
    grid2.set(x,y,0)

final_answer = np.sum(np.array(grid2.grid))
print(final_answer)

1935


## trace the cube back to see how it started


In [130]:
rev = list(reversed(ans.history))
rev_steps = []
for i in range(len(rev)-1):
    x1, y1 = rev[i]
    x2, y2 = rev[i+1]

    rev_steps.append((x2-x1, y2-y1))

In [131]:
dirs_dict = {
    (0,1) : 0,
    (1,0) : 1,
    (0,-1): 2,
    (-1,0): 3
}

rev_dirs = [dirs_dict[step] for step in rev_steps]
rev_cube = copy.deepcopy(ans.cube)
for dir in rev_dirs:
    rev_cube.roll(dir)

## starting cube, steps, final answer

In [132]:
print(final_answer)
print(rev_cube)
print(ans.history)

1935

        	-9.0
        9.0	9.0	-3.0
        	5.0
        	7.0
        
[(0, 0), (0, 1), (1, 1), (2, 1), (2, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 5), (2, 4), (1, 4), (0, 4), (0, 3), (1, 3), (2, 3), (2, 2), (3, 2), (3, 1), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (4, 2), (3, 2), (3, 3), (3, 4), (4, 4), (5, 4), (5, 5)]


<img src="https://www.janestreet.com/puzzles/die-agony.png" alt="drawing" width="200"/>